In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from math import *

def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, real_dim, real_dim, 1), name="inputs_real")
    inputs_z = tf.placeholder(tf.float32, (None, z_dim))
    return inputs_real, inputs_z

def generator(z, input_size=64, reuse=False,  alpha=0.01):
    with tf.variable_scope('generator', reuse=reuse):
        # Hidden layer 1
        h1 = tf.layers.dense(z, (input_size//8)*(input_size//8)*256, activation=None)
        h1 = tf.reshape(h1, [-1, (input_size//8), (input_size//8), 256])
        h1 = tf.maximum(h1, alpha*h1)
        h1 = tf.nn.dropout(h1, 0.4)
        # Upsampling-convolution layer 1
        upsample1 = tf.image.resize_nearest_neighbor(h1, [(input_size//4), (input_size//4)])
        conv1_1 = tf.layers.conv2d(inputs=upsample1, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_1")
        conv1_1 = tf.maximum(conv1_1, alpha*conv1_1)
        norm1_1 = tf.contrib.layers.batch_norm(inputs = conv1_1)
        conv1_2 = tf.layers.conv2d(inputs=norm1_1, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_2")
        conv1_2 = tf.maximum(conv1_2, alpha*conv1_2)
        norm1_2 = tf.contrib.layers.batch_norm(inputs = conv1_2)
        conv1_3 = tf.layers.conv2d(inputs=norm1_2, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_3")
        conv1_3 = tf.maximum(conv1_3, alpha*conv1_3)
        norm1_3 = tf.contrib.layers.batch_norm(inputs = conv1_3)
        conv1_4 = tf.layers.conv2d(inputs=norm1_3, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_4")
        conv1_4 = tf.maximum(conv1_4, alpha*conv1_4)
        norm1_4 = tf.contrib.layers.batch_norm(inputs = conv1_4)
        
        # Upsampling-convolution layer 2
        upsample2 = tf.image.resize_nearest_neighbor(norm1_4, [input_size//2, input_size//2])
        conv2_1 = tf.layers.conv2d(inputs=upsample2, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_1")
        conv2_1 = tf.maximum(conv2_1, alpha*conv2_1)
        norm2_1 = tf.contrib.layers.batch_norm(inputs = conv2_1)
        conv2_2 = tf.layers.conv2d(inputs=norm2_1, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_2")
        conv2_2 = tf.maximum(conv2_2, alpha*conv2_2)
        norm2_2 = tf.contrib.layers.batch_norm(inputs = conv2_2)
        conv2_3 = tf.layers.conv2d(inputs=norm2_2, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_3")
        conv2_3 = tf.maximum(conv2_3, alpha*conv2_3)
        norm2_3 = tf.contrib.layers.batch_norm(inputs = conv2_3)
        conv2_4 = tf.layers.conv2d(inputs=norm2_3, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_4")
        conv2_4 = tf.maximum(conv2_4, alpha*conv2_4)
        norm2_4 = tf.contrib.layers.batch_norm(inputs = conv2_4)
        
        # Upsampling-convolution layer 3
        upsample3 = tf.image.resize_nearest_neighbor(norm2_4, [input_size, input_size])
        conv3_1 = tf.layers.conv2d(inputs=upsample3, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_1")
        conv3_1 = tf.maximum(conv3_1, alpha*conv3_1)
        norm3_1 = tf.contrib.layers.batch_norm(inputs = conv3_1)
        conv3_2 = tf.layers.conv2d(inputs=norm3_1, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_2")
        conv3_2 = tf.maximum(conv3_2, alpha*conv3_2)
        norm3_2 = tf.contrib.layers.batch_norm(inputs = conv3_2)
        conv3_3 = tf.layers.conv2d(inputs=norm3_2, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_3")
        conv3_3 = tf.maximum(conv3_3, alpha*conv3_3)
        norm3_3 = tf.contrib.layers.batch_norm(inputs = conv3_3)
        conv3_4 = tf.layers.conv2d(inputs=norm3_3, filters=1, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_4")
        
        # Logits and tanh output
        logits = conv3_4
        out = tf.nn.tanh(logits)
        tf.summary.image("Generated_image", (tf.reshape(out, [-1, input_size, input_size, 1])+1) * 0.5, 3)
        
        return out, logits
        
        
# Size of input image to discriminator
input_size = 64 # 64x64 Face images
# Size of latent vector to generator
z_size = 100
# Leak factor for leaky ReLU
alpha = 0.01
# Label smoothing 
smooth = 0.1

tf.reset_default_graph()
# Create our input placeholders
input_real, input_z = model_inputs(input_size, z_size)

# Generator network here
g_model, g_logits = generator(input_z, input_size,reuse=False,  alpha=alpha)
# g_model is the generator output

t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith("generator")]

logging_step = 50
batch_size = 32
epochs = 20
samples = []
losses = []
saver = tf.train.Saver(var_list = g_vars)
with tf.Session() as sess:
    saver.restore(sess, 'Weights/ganFaces1')
    N_images = int(input('Enter the number of images to generate: '))
    plt.figure(figsize=(10, ((N_images // 5)+1)))
    for n in range(N_images):
        batch_z = np.random.uniform(-1, 1, size=(1, z_size))
        plt.subplot(((N_images // 5) + 1), 5, n+1)
        generated_pic = g_model.eval(feed_dict={input_z: batch_z}).reshape((input_size, input_size))
        plt.imshow(generated_pic, cmap='gray')
        plt.xticks([])
        plt.yticks([])
        
        
